In [ ]:
# Seed value
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ISPR/modules/')
from metrics import dice_coef
from fcn import getFCN
from losses import dice_loss, tversky_Loss

# Load Data

In [ ]:
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
SIZE = 128
image_dataset = []
mask_dataset = []

image_directory = "/content/drive/MyDrive/ISPR/immagini_x_"+str(SIZE)+"/"
mask_directory = "/content/drive/MyDrive/ISPR/immagini_y_"+str(SIZE)+"/"

In [ ]:
#Images
images = os.listdir(image_directory)
images.sort()
for i, image_name in enumerate(images): 
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory+image_name, 0)
        image = Image.fromarray(image)
        image_dataset.append(np.array(image))

#Masks
masks = os.listdir(mask_directory)
masks.sort()
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'jpg' ):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        mask_dataset.append(np.array(image))

In [ ]:
#Expand the dimension of the image dataset to include also the number of channels
image_dataset = np.expand_dims(image_dataset,3)
#Expand the dimension of the mask dataset to include also the number of channels
#and threshold the data to obtain feasible masks
mask_dataset = np.clip(np.expand_dims((np.array(mask_dataset)),3) - 200.0,0,1)

In [ ]:
#Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.1, random_state = 0)

In [ ]:
#sample data
import random
import numpy as np
image_number = random.randint(0, len(X_train))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(np.reshape(image_dataset[image_number], (SIZE, SIZE)), cmap='gray')
plt.subplot(122)
plt.imshow(np.reshape(mask_dataset[image_number], (SIZE, SIZE)), cmap='gray')
plt.show()

# Model

In [ ]:
path = [32,64,128,128]
addMask = [0,1,1,0]
bridge = 256
dropout = 0.1
#loss = 'binary_crossentropy'
loss = dice_loss

model = getFCN(SIZE, path, addMask, bridge, dropout, loss)

In [ ]:
history = model.fit(X_train, y_train, 
                    batch_size = 16,  
                    verbose=1, 
                    epochs=5,  
                    validation_split  = 0.1,
                    shuffle=False)


# Plots

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
#Model Accuracy
fig, ax = plt.subplots()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

ax.set_xticks([0, 4, 9, 14, 19])
ax.set_xticklabels(['1', '5', '10', '15' , '20'])

plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Model Dice Coefficient
fig, ax = plt.subplots()
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('Model Dice Coefficient')
plt.ylabel('Dice Coefficient')
plt.xlabel('Epoch')

ax.set_xticks([0, 4, 9, 14, 19])
ax.set_xticklabels(['1', '5', '10', '15' , '20'])

plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Model Loss
fig, ax = plt.subplots()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

ax.set_xticks([0, 4, 9, 14, 19])
ax.set_xticklabels(['1', '5', '10', '15' , '20'])

plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Test trained model
image_number = random.randint(0, len(X_test))
plt.figure(figsize=(12, 6))
plt.subplot(131)
plt.imshow(np.reshape(X_test[image_number], (SIZE, SIZE)), cmap='gray')
plt.subplot(132)
plt.imshow(np.reshape(y_test[image_number], (SIZE, SIZE)), cmap='gray')
plt.subplot(133)
pred = model.predict(X_test[image_number].reshape(1,SIZE,SIZE,1))
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
plt.imshow(np.reshape(pred, (SIZE, SIZE)), cmap='gray')
plt.show()

The dice coefficient is: 0.03298434624513995
